In [ ]:
# %% [markdown]
# # 通用城市天气分析系统
# **通过修改 .env 文件即可切换任意城市**

# %% [code]
# 单元格1：动态加载配置
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 添加项目根目录
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# 导入通用模块
from src.config import config
from src.fetcher import UniversalWeatherFetcher

print(config)  # 显示当前配置


In [ ]:

# %% [code]
# 单元格2：验证并初始化
if config.validate():
    print("✅ 配置验证通过！")
    fetcher = UniversalWeatherFetcher()
else:
    print("❌ 配置验证失败，请检查 .env 文件")
    


In [ ]:
# %% [code]
# 单元格3：获取当前配置城市的天气
print(f"\n正在分析 {config.CITY_DISPLAY_NAME} 的天气...")
current_data = fetcher.get_current_weather()

if current_data:
    parsed_data = UniversalWeatherFetcher.parse_weather_data(
        current_data, 
        config.CITY_DISPLAY_NAME
    )
    
    print(f"\n📍 {config.CITY_DISPLAY_NAME} 实时天气")
    print("=" * 40)
    for key in ['当前温度(℃)', '天气状况', '湿度(%)', '风速(m/s)', '舒适度评级']:
        if key in parsed_data:
            print(f"{key:>10}: {parsed_data[key]}")
    print("=" * 40)


In [ ]:

# %% [code]
# 单元格4：动态切换城市（无需修改代码）
def analyze_city(city_name=None, country_code=None, display_name=None):
    """动态分析指定城市"""
    if not city_name:
        city_name = config.CITY
    if not country_code:
        country_code = config.COUNTRY
    if not display_name:
        display_name = city_name
    
    print(f"\n🔍 正在分析 {display_name} ...")
    
    data = fetcher.get_current_weather(city_name, country_code)
    
    if data:
        parsed = UniversalWeatherFetcher.parse_weather_data(data, display_name)
        
        # 保存到文件（按城市命名）
        import pandas as pd
        df = pd.DataFrame([parsed])
        filename = f"data/raw/{city_name.lower()}_weather.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"💾 数据已保存到: {filename}")
        
        return parsed
    return None

# 示例：动态分析不同城市
cities_to_analyze = [
    {"city": "Beijing", "country": "CN", "display_name": "北京"},
    {"city": "Shanghai", "country": "CN", "display_name": "上海"},
    {"city": "Guilin", "country": "CN", "display_name": "桂林"},
]

all_results = []
for city_info in cities_to_analyze:
    result = analyze_city(**city_info)
    if result:
        all_results.append(result)


In [ ]:

# %% [code]
# 单元格5：多城市对比可视化
if all_results:
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    
    # 转换为DataFrame
    df_comparison = pd.DataFrame(all_results)
    
    # 创建对比图表
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 温度对比
    cities = df_comparison['城市']
    temps = df_comparison['当前温度(℃)']
    
    bars = axes[0, 0].bar(cities, temps, color=['coral', 'skyblue', 'lightgreen'])
    axes[0, 0].set_title('城市温度对比', fontweight='bold')
    axes[0, 0].set_ylabel('温度 (℃)')
    axes[0, 0].set_ylim(0, max(temps) + 5)
    
    # 添加温度值标签
    for bar, temp in zip(bars, temps):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                       f'{temp}℃', ha='center', va='bottom')
    
    # 湿度对比
    humidity = df_comparison['湿度(%)']
    wedges, texts, autotexts = axes[0, 1].pie(
        humidity, 
        labels=cities,
        autopct='%1.1f%%',
        colors=['lightcoral', 'lightblue', 'lightgreen']
    )
    axes[0, 1].set_title('城市湿度对比', fontweight='bold')
    
    # 风速对比
    wind_speed = df_comparison['风速(m/s)']
    x_pos = np.arange(len(cities))
    axes[1, 0].bar(x_pos, wind_speed, color='orange', alpha=0.7)
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(cities)
    axes[1, 0].set_title('城市风速对比', fontweight='bold')
    axes[1, 0].set_ylabel('风速 (m/s)')
    
    # 舒适度评级
    comfort = df_comparison['舒适度评级']
    comfort_colors = []
    for rating in comfort:
        if '舒适' in rating:
            comfort_colors.append('green')
        elif '寒冷' in rating or '炎热' in rating:
            comfort_colors.append('red')
        else:
            comfort_colors.append('yellow')
    
    axes[1, 1].barh(cities, [1, 1, 1], color=comfort_colors, edgecolor='black')
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_xticks([])
    axes[1, 1].set_title('舒适度评级', fontweight='bold')
    
    # 添加评级文本
    for i, (city, rating) in enumerate(zip(cities, comfort)):
        axes[1, 1].text(0.5, i, rating, ha='center', va='center', 
                       fontweight='bold', color='white' if rating != '较为舒适' else 'black')
    
    plt.suptitle(f'多城市天气对比分析\n生成时间: {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")}', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


# %% [markdown]
# ## 如何切换城市
# 
# 要分析其他城市，只需修改 `.env` 文件中的以下配置：
# 
# ```env
# WEATHER_CITY=Shanghai      # 改为目标城市英文名
# CITY_DISPLAY_NAME=上海     # 改为目标城市显示名
# WEATHER_COUNTRY=CN         # 国家代码
# ```
# 
# 修改后重新运行此笔记本即可！


In [ ]:

# %% [code]
# 单元格6：生成城市切换工具函数
def quick_city_analysis(city_name, country_code='CN', display_name=None):
    """
    快速分析指定城市
    示例: quick_city_analysis('Guangzhou', 'CN', '广州')
    """
    if not display_name:
        display_name = city_name
    
    print(f"🚀 快速分析: {display_name}")
    print("-" * 30)
    
    data = fetcher.get_current_weather(city_name, country_code)
    
    if data:
        parsed = UniversalWeatherFetcher.parse_weather_data(data, display_name)
        
        # 显示关键信息
        info_template = """
城市: {城市}
温度: {当前温度(℃)}℃ (体感: {体感温度(℃)}℃)
天气: {天气状况} ({详细描述})
湿度: {湿度}%
风速: {风速(m/s)} m/s
舒适度: {舒适度评级}
更新时间: {更新时间}
        """.format(
            城市=parsed['城市'],
            当前温度=parsed['当前温度(℃)'],
            体感温度=parsed['体感温度(℃)'],
            天气状况=parsed['天气状况'],
            详细描述=parsed['详细描述'],
            湿度=parsed['湿度(%)'],
            风速=parsed['风速(m/s)'],
            舒适度评级=parsed['舒适度评级'],
            更新时间=parsed['更新时间']
        )
        
        print(info_template)
        return parsed
    
    return None

# 示例使用
print("示例：快速分析不同城市")
print("=" * 40)
quick_city_analysis('Beijing', 'CN', '北京')
print()
quick_city_analysis('Hangzhou', 'CN', '杭州')